In [1]:
import keras
import functools

# residual unit

## conv2D layer

het punt hiervan is zodat je niet telkens dit moet doen:

```python
keras.layers.SeparableConv2D(filters=64, kernel_size=(3,3), padding='same')
keras.layers.SeparableConv2D(filters=128, kernel_size=(3,3), padding='same')
```

maar gewoon:

```python
DefaultSeparableConv(filters=64)
DefaultSeparableConv(filters=128)
```

In [ ]:
DefaultConv = functools.partial(
    keras.layers.Conv2D, kernel_size=(1, 1), strides=(1, 1), padding="same", activation="relu"
    )

In [ ]:
DefaultSeparableConv = functools.partial(
    keras.layers.SeparableConv2D, kernel_size=(3,3), padding='same'
)

## init

In [ ]:
def __init__(self, filters, use_max_pool=False, **kwargs):
    super().__init__(**kwargs)

    self.main_path = [
        keras.layers.ReLU(),
        DefaultSeparableConv(filters=filters),
        keras.layers.ReLU(),
        DefaultSeparableConv(filters=filters),
    ]

    self.skip_path = []
    if not use_max_pool:
      self.main_path.append(keras.layers.ReLU())
      DefaultSeparableConv(filters=filters)
    else:
      self.main_path.append(keras.layers.MaxPool2D(pool_size=(3,3), padding='same', strides=(2,2)))
      self.skip_path = [
          keras.layers.Conv2D(filters=filters, kernel_size=(1,1), padding='same', strides=(2,2))
      ]


## call

In [ ]:
def call(self, input):
    skip = input
    for layer in self.main_path:
        input = layer(input)

    for layer in self.skip_path:
        skip = layer(skip)

    return input + skip

In [ ]:
def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
        Z = layer(Z)
    
    skip_Z = inputs
    for layer in self.skip_layers:
        skip_Z = layer(skip_Z)
    
    return self.activation(Z + skip_Z)

![](../image/foto1.png)
![](../image/foto2.png)

this is an example from the book

In [ ]:
DefaultConv = functools.partial( keras.layers.Conv2D, kernel_size=(1, 1), strides=(1, 1), padding="same", activation="relu")

class InceptionModule(keras.layers.Layer):
    def __init__(self, filters11, filters33_reduce, filters33,
                 filters55_reduce , filters55, filters_pool_proj,
                 use_batch_norm=True, **kwargs):
        super().__init__(**kwargs)
        self.conv11 = DefaultConv(filters=filters11)
        self.conv33 = DefaultConv(filters=filters33, kernel_size=(3,3))
        self.conv33_reduce = DefaultConv(filters=filters33_reduce)
        self.conv55 = DefaultConv(filters=filters55, kernel_size=(5,5))
        self.conv55_reduce = DefaultConv(filters=filters55_reduce)
        self.conv_pool_proj = DefaultConv(filters=filters_pool_proj)
        self.max_pool = keras.layers.MaxPool2D(pool_size=(3,3), strides=1, padding="same")

        self.use_batch_norm = use_batch_norm
        if use_batch_norm:
          self.batch_norm = keras.layers.BatchNormalization()

    def call(self, inputs):
        path1 = self.conv11(inputs)
        path2 = self.conv33_reduce(inputs)
        path2 = self.conv33(path2)
        path3 = self.conv55_reduce(inputs)
        path3 = self.conv55(path3)
        path4 = self.max_pool(inputs)
        path4 = self.conv_pool_proj(path4)

        result = keras.layers.Concatenate()([path1, path2, path3, path4])

        if self.use_batch_norm:
          return self.batch_norm(result)
        return result